In [216]:
from __future__ import division
import vertica_python
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from vertica_python import connect
import StringIO
from StringIO import StringIO
from datetime import date, timedelta as td
import xlsxwriter
import os
import json
import urllib2
import base64
import datetime
import re
import csv
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

In [217]:
import argparse

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [218]:
def get_service(api_name, api_version, scope, client_secrets_path):
  """Get a service that communicates to a Google API.

  Args:
    api_name: string The name of the api to connect to.
    api_version: string The api version to connect to.
    scope: A list of strings representing the auth scopes to authorize for the
      connection.
    client_secrets_path: string A path to a valid client secrets file.

  Returns:
    A service that is connected to the specified API.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      client_secrets_path, scope=scope,
      message=tools.message_if_missing(client_secrets_path))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage(api_name + '.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  service = build(api_name, api_version, http=http)

  return service

In [219]:
def get_ga_data_multi_campaign(service, course_enrolls_dict):

    out = service.data().ga().get(
        ids='ga:' + '86300562',
        start_date=str(course_enrolls_dict['start_date']),
        end_date=str(course_enrolls_dict['end_date']),
        max_results=10000,
        metrics=','.join(course_enrolls_dict['metric_list']),
        dimensions=','.join(course_enrolls_dict['dimension_list']),
        filters=','.join(course_enrolls_dict['filter_list'])).execute()
                            
    col_list = []
                            
    for i in np.arange(0,len(out['columnHeaders'])):
        line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
        col_list.append(line)
    return pd.DataFrame(out['rows'],columns=col_list)


In [220]:
def get_ga_data(service, course_enrolls_dict):

    out = service.data().ga().get(
        ids='ga:' + '86300562',
        start_date=str(course_enrolls_dict['start_date']),
        end_date=str(course_enrolls_dict['end_date']),
        max_results=10000,
        metrics=','.join(course_enrolls_dict['metric_list']),
        dimensions=','.join(course_enrolls_dict['dimension_list']),
        filters=';'.join(course_enrolls_dict['filter_list'])).execute()
                            
    col_list = []
                            
    for i in np.arange(0,len(out['columnHeaders'])):
        line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
        col_list.append(line)
    return pd.DataFrame(out['rows'],columns=col_list)


In [238]:
scope = ['https://www.googleapis.com/auth/analytics.readonly']
service = get_service('analytics', 'v3', scope, 'client_secrets.json')

In [239]:
enrollments_transactions_dict_newsletter = {
    'filter_list': [
'ga:campaign==studentnewsletter-20161115',
'ga:eventLabel=~course-'
],
    'dimension_list': [
'ga:campaign',
'ga:eventLabel',
],
    'metric_list': [
'ga:goal8Completions',
'ga:transactions',
'ga:transactionRevenue'
],
    'start_date': '2016-10-31',
    'end_date': '2016-12-01',
}

In [262]:
newsletter_df = pd.DataFrame()

for i in pd.date_range('2016-11-01', '2016-12-01'):
    
    out = service.data().ga().get(
    ids='ga:' + '86300562',
    start_date=str(i.date()),
    end_date=str(i.date()),
    max_results=10000,
    metrics='ga:sessions,ga:goal8Completions,ga:transactions,ga:transactionRevenue',
    dimensions='ga:campaign, ga:eventLabel',
    filters='ga:campaign==studentnewsletter-20161115;ga:eventLabel=~course-',
    samplingLevel = 'HIGHER_PRECISION').execute()

    col_list = []

    for i in np.arange(0,len(out['columnHeaders'])):
        line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
        col_list.append(line)
    try:
        newsletter_df = newsletter_df.append(pd.DataFrame(out['rows'],columns=col_list))
    except:
        len(out) == 0

In [263]:
newsletter_df = newsletter_df.rename(columns={'eventLabel': 'course_id','goal8Completions':'enrollments'})
newsletter_df[['sessions','enrollments','transactions','transactionRevenue']] = newsletter_df[['sessions','enrollments','transactions','transactionRevenue']].apply(pd.to_numeric)

In [286]:
newsletter_df = newsletter_df.groupby('course_id').sum().sort_values('enrollments', ascending=False)

In [288]:
newsletter_df.to_csv('./../script_output/bisupport-6_newsletter.csv')

In [240]:
enrollments_transactions_df = get_ga_data(service, enrollments_transactions_dict_newsletter)

In [246]:
enrollments_transactions_df[['enrollments','transactions','transactionRevenue']] = enrollments_transactions_df[['enrollments','transactions','transactionRevenue']].apply(pd.to_numeric)

In [247]:
#filter dataframe if all rows are zero
enrollments_transactions_df = enrollments_transactions_df[
    (enrollments_transactions_df[['enrollments','transactions','transactionRevenue']]!=0).any(1)]

In [287]:
enrollments_transactions_df.head()

,campaign,course_id,enrollments,transactions,transactionRevenue
24,studentnewsletter-20161115,course-v1:ASUx+TGM540x+1T2017,439,0,0.0
31,studentnewsletter-20161115,course-v1:AmnestyInternationalX+Rights2x+3T2016,120,0,0.0
51,studentnewsletter-20161115,course-v1:BerkeleyX+ColWri2.1x+3T2016,120,0,0.0
52,studentnewsletter-20161115,course-v1:BerkeleyX+ColWri2.2x+1T2017,200,0,0.0
54,studentnewsletter-20161115,course-v1:BerkeleyX+EE40LX+2T2015,40,0,0.0


In [190]:
enrollments_transactions_df.to_csv('./../script_output/bisupport-6_newsletter.csv')

In [126]:
out = service.data().ga().get(
    ids='ga:' + '86300562',
    start_date=str('2016-11-15'),
    end_date=str('2016-11-30'),
    max_results=10000,
    metrics='ga:sessions,ga:goal8Completions,ga:transactions,ga:transactionRevenue',
    dimensions='ga:campaign',
    filters='ga:campaign==leadnurture-ai-20161117,ga:campaign==MicroMastersLeadGenMembers-20161117').execute()

col_list = []

for i in np.arange(0,len(out['columnHeaders'])):
    line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
    col_list.append(line)

mm_email_df2 = pd.DataFrame(out['rows'],columns=col_list)

In [234]:
micromaster_email_list = [
#'ga:campaign==leadnurture-projectmanagement-20161117',
'ga:campaign==leadnurture-ai-20161117',
'ga:campaign==MicroMastersLeadGenMembers-20161117',

]

In [235]:
mm_df = pd.DataFrame()

for i in pd.date_range('2016-11-01', '2016-12-01'):
    
    out = service.data().ga().get(
    ids='ga:' + '86300562',
    start_date=str(i.date()),
    end_date=str(i.date()),
    max_results=10000,
    metrics='ga:sessions,ga:goal8Completions,ga:transactions,ga:transactionRevenue',
    dimensions='ga:campaign',
    filters='ga:sourceMedium==sailthru / email;' + ','.join(micromaster_email_list),
    samplingLevel = 'HIGHER_PRECISION').execute()

    col_list = []

    for i in np.arange(0,len(out['columnHeaders'])):
        line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
        col_list.append(line)
    try:
        mm_df = mm_df.append(pd.DataFrame(out['rows'],columns=col_list))
    except:
        len(out) == 0

In [236]:
mm_df[['sessions','goal8Completions','transactions','transactionRevenue']] = mm_df[['sessions','goal8Completions','transactions','transactionRevenue']].apply(pd.to_numeric)
mm_df = mm_df.groupby('campaign').sum()
mm_df = mm_df.rename(columns={'goal8Completions':'enrollments'})

In [237]:
mm_df

,sessions,enrollments,transactions,transactionRevenue
campaign,,,,
MicroMastersLeadGenMembers-20161117,1377,68,3,442.923162
leadnurture-ai-20161117,405,45,0,0.000000


In [198]:
mm_df.to_csv('./../script_output/bisupport-6_mm.csv')